## ※ Business analytics tutorial- Semi-supervised Learning
---
본 튜토리얼은 고려대학교 강필성 교수님의 Business analytics 수업의 내용을 정리한 튜토리얼 코드 입니다.  
튜토리얼에서는 Semi-supervised Learning의 대표적인 방법론에 대한 코드와 간단한 설명을 다룹니다.  
또한 튜토리얼은 코드는 microsoft에서 구현한 Semi-supervised Learning tutorial code를 활용하여 작성하였습니다.

출처: https://github.com/microsoft/Semi-supervised-learning

## ※ Microsoft tutorial code 설명
---
Microsoft에서 재공하는 tutorial code의 경우 Pi-Model, Mean-Teacher, FixMatch, FlexMatch, AdaMatch 등 총 14가지의 Semi-supervised learning 알고리즘과 Superviesd, Fully-supervised learning을 제공합니다.  
또한 yaml 파일을 통해 각 알고리즘, 데이터셋 별로 하이퍼파라미터를 저장해 놓았습니다.  
본 튜토리얼에서는 Microsoft에서 재공하는 tutorial을 통해 각각의 Semi-supervised 방법론 별로 labeled data의 비율을 변경하였을 때의 성능에 대한 차이를 분석하고자 합니다. 

In [1]:
# Microsoft Semi Supervised learning 라이브러리 설치

!pip install semilearn==0.3.8a


In [1]:
# GPU 메모리 상태 확인

!nvidia-smi

Tue Dec 27 08:16:55 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.86.01    Driver Version: 515.86.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| 35%   30C    P8     3W / 260W |    101MiB / 11264MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# 라이브러리 호출

import semilearn
from semilearn import get_dataset, get_data_loader, get_net_builder, get_algorithm, get_config, Trainer

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 원하는 알고리즘의 config 구성
Microsoft Semi-supervised-learning 패키지의 경우 github code에 각 알고리즘 별 parameter를 yaml 파일로 제공합니다. 각 알고리즘의 parameter를 최대한 그대로 사용하지만, 개인 pc와 시간으로 인해 각 모델별 epoch을 10, batch size를 16(FixMatch, FlexMacth의 경우 8)으로 변경하였습니다. 

또한 pretrain model을 vit_tiny_patch2_32를 사용하였습니다. 


In [7]:
import yaml

def set_config(algorithm, num_labels ,epochs = 10):
    legislators = yaml.safe_load(open(f"config/{algorithm}/{algorithm}_cifar10_{num_labels}_0.yaml",encoding='UTF-8'))
    # 실험 시 0번 gpu 사용, 분산 사용 X
    
    legislators['gpu'] = 0
    legislators['use_pretrain'] = True
    legislators['pretrain_path'] = 'https://github.com/microsoft/Semi-supervised-learning/releases/download/v.0.0.0/vit_tiny_patch2_32_mlp_im_1k_32.pth'
    legislators['distributed'] = False
    legislators['epoch'] = epochs
    legislators['batch_size'] = 16
    legislators['num_workers'] = 4
    legislators['net'] = 'vit_tiny_patch2_32'
    legislators['num_labels'] = num_labels
    legislators['num_train_iter'] = int(50000/legislators['batch_size'] * epochs) # len(data) = 50,000
    
    config = get_config(legislators)
    
    return config

### 알고리즘 설명

본 튜토리얼에서는 강의 내용에서 다룬 consitency regularization 방법론인 Pi-model, Mean teacher와 holistic 방법론인 MixMatch, ReMixMatch, FixMatch, FlexMatch에 대한 실험을 진행하였습니다. 각 방법론에 대한 설명은 다음과 같습니다.  

- Consitency regularization

    - Pi-model  
      
      <img src="ssl_pictures/pi_model.png" width="1213px" height="200px" title="pi model">  
      
      Pi-model의 경우 위의 그림과 같이 하나의 FFN에 두 번의 perturbation을 적용합니다.  
      또한 Ladder network와는 다르게 latent vector가 아닌 ouput vector가 유사하게 산출되도록 학습을 진행합니다.  
      즉 labeled data를 통해 cross entropy를 활용한 supervised loss를 산출하고, unlabeled data를 통해 mean squared error(MSE)를 통해 consistency loss를 산출합니다.  
      최종적으로 두개의 loss를 일차결합하여 최종 loss function을 산출하고, loss가 최소화 되도록 학습을 진행합니다.
      
        
    - Mean Teacher   
      
      <img src="ssl_pictures/mean_teacher.png" width="1213px" height="547px" title="mean teacher">  

      Mean Teacher의 경우 teacher model과 student model을 구분하고, student model은 labeled data와 unlabeled data를 모두 사용하고, teacher model은 unlabeled data만을 사용합니다. 따라서, student model은 cross entropy 기반의 supervised loss, MSE 기반의 consistency loss 산출에 사용되고, teacher model은 consistency loss에만 사용됩니다. 또한 batch 단위로 EMA를 진행하여 Teacher model의 가중치를 student model의 가중치로 보정합니다.  
      

- Holistic method

    - MixMatch  

      <img src="ssl_pictures/MixMatch.png" width="1213px" height="433px" title="MixMatch">  
        
      MixMatch의 경우 unlabeled data를 Augmentation을 활용하여 semi-supervised learning을 진행합니다. Augmentation을 진행한 sample들의 output의 평균을 활용하여 label을 guessing하고 나서 softmax temperature를 적용하여 Entropy minimization을 수행하고 Mixup을 적용합니다.  
  

    - ReMixMatch  

      <img src="ssl_pictures/RemixMatch.png" width="1213px" height="818px" title="ReMixMatch">  
        
      ReMixMatch 역시 전반적인 진행방식은 MixMatch와 매우 유사합니다. ReMixMatch의 경우 MixMatch에서는 진행하지 않은 unlabeled data와 labeled data의 분포를 유사하게 만드는 distribution alignment와 strong augmentation이 추가됩니다. Strong augmentation을 통해 얻은 sample의 경우 신뢰도가 weak augmentation에 비해 낮을 확률이 높기 때문에 label을 guessing하는 과정에서는 사용하지 않습니다.  

    - FixMatch  

      <img src="ssl_pictures/Fixmatch.png" width="1213px" height="490px" title="FixMatch">  
        
      FixMatch의 경우도 전반적인 진행방식은 위에서 언급한 MixMatch, ReMixMatch와 유사합니다. 차이점은 weak augmentation sample의 prediction의 신뢰도가 충분히 높을 경우 해당 pseudo-label을 strong augmentation sample의 pseudo-label로 사용하여 학습을 진행하는 것 입니다.  
        

    - FlexMatch  

      <img src="ssl_pictures/FlexMatch.png" width="1213px" height="308px" title="ReMixMatch">  
        
      FlexMatch의 경우 FixMatch를 조금 더 개선하여 만든 알고리즘입니다. Fixmatch의 경우 모든 class에 동일한 confidence 기준을 설정하였으나, FlexMatch의 경우 각 class마다 고유의 confidence 기준을 설정하여 학습을 진행합니다. 즉, 예측이 어려운 class의 경우 조금 더 낮은 confidence를 적용하고 예측이 쉬운 class의 경우 더 높은 confidence를 적용하여 학습을 진행합니다.  
     

In [4]:
# 알고리즘 및 num_label 설정

algorithm_list = ['pimodel','meanteacher','mixmatch','remixmatch','fixmatch','flexmatch']

num_labels_list = [40] # cifar 10 dataset은 각각의 class가 train 5,000개, test 1,000개로 구성 -> class당 4개의 label data 사용

In [4]:
# 지정한 config에 맞는 dataset, train_lb_loader, train_ulb_loader, eval_loader를 생성하는 함수

def dataloader(algorithm, num_labels):
    config = set_config(algorithm,num_labels)
    dataset_dict = get_dataset(config, config.algorithm, config.dataset, config.num_labels, config.num_classes, data_dir=config.data_dir)
    train_lb_loader = get_data_loader(config, dataset_dict['train_lb'], config.batch_size)
    train_ulb_loader = get_data_loader(config, dataset_dict['train_ulb'], int(config.batch_size * config.uratio))
    eval_loader = get_data_loader(config, dataset_dict['eval'], config.eval_batch_size)
    
    return train_lb_loader, train_ulb_loader, eval_loader

In [5]:
# 지정한 config에 대해 train을 진행하는 함수 설정
import gc
import torch

def train(algorithm, num_labels, train_lb_loader, train_ulb_loader, eval_loader, epochs):
    # Cache 제거를 통해 메모리 최적화
    gc.collect()
    torch.cuda.empty_cache()
    
    config = set_config(algorithm,num_labels,epochs)
    
    algorithm = get_algorithm(config,  get_net_builder(config.net, from_name=False), tb_log=None, logger=None)
    
    trainer = Trainer(config, algorithm)
    trainer.fit(train_lb_loader, train_ulb_loader, eval_loader)
    
    return trainer

In [16]:
# 10 epcoh으로 실험 진행

epochs = 10

for algorithm in algorithm_list:
    for num_labels in num_labels_list:
        train_lb_loader, train_ulb_loader, eval_loader = dataloader(algorithm, num_labels)
        trainer = train(algorithm, num_labels, train_lb_loader, train_ulb_loader, eval_loader, epochs)
        globals()["eval_dict_{}_{}".format(algorithm, num_labels)] = trainer.evaluate(eval_loader)
        globals()["y_pred_{}_{}".format(algorithm, num_labels)], globals()["y_logit_{}_{}".format(algorithm, num_labels)]= trainer.predict(eval_loader)

/bin/sh: 1: netstat: not found


Files already downloaded and verified
lb count: [4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
ulb count: [5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000]
Files already downloaded and verified


/bin/sh: 1: netstat: not found


Files already downloaded and verified
lb count: [4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
ulb count: [5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000]
Files already downloaded and verified
unlabeled data number: 50000, labeled data number 40
Create train and test data loaders
[!] data loader keys: dict_keys(['train_lb', 'train_ulb', 'eval'])
_IncompatibleKeys(missing_keys=['head.weight', 'head.bias'], unexpected_keys=[])
Create optimizer and scheduler
Epoch: 0


[2022-12-25 09:50:36,934 INFO] confusion matrix
[2022-12-25 09:50:36,935 INFO] [[0.201 0.068 0.119 0.116 0.017 0.038 0.082 0.028 0.197 0.134]
 [0.068 0.161 0.012 0.073 0.008 0.021 0.203 0.009 0.114 0.331]
 [0.047 0.074 0.089 0.075 0.124 0.094 0.161 0.152 0.083 0.101]
 [0.013 0.101 0.048 0.066 0.073 0.071 0.348 0.146 0.035 0.099]
 [0.015 0.061 0.066 0.077 0.277 0.082 0.118 0.187 0.057 0.06 ]
 [0.012 0.063 0.046 0.072 0.074 0.054 0.36  0.164 0.046 0.109]
 [0.004 0.048 0.043 0.053 0.308 0.043 0.217 0.229 0.027 0.028]
 [0.021 0.11  0.038 0.096 0.038 0.068 0.305 0.106 0.057 0.161]
 [0.133 0.093 0.077 0.103 0.004 0.025 0.102 0.013 0.205 0.245]
 [0.082 0.134 0.011 0.067 0.011 0.034 0.196 0.019 0.107 0.339]]
[2022-12-25 09:50:36,937 INFO] evaluation metric
[2022-12-25 09:50:36,937 INFO] acc: 0.1715
[2022-12-25 09:50:36,938 INFO] precision: 0.1793
[2022-12-25 09:50:36,938 INFO] recall: 0.1715
[2022-12-25 09:50:36,938 INFO] f1: 0.1682


...

Files already downloaded and verified
lb count: [4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
ulb count: [5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000]
Files already downloaded and verified


/bin/sh: 1: netstat: not found


Files already downloaded and verified
lb count: [4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
ulb count: [5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000]
Files already downloaded and verified
unlabeled data number: 50000, labeled data number 40
Create train and test data loaders
[!] data loader keys: dict_keys(['train_lb', 'train_ulb', 'eval'])
_IncompatibleKeys(missing_keys=['head.weight', 'head.bias'], unexpected_keys=[])
Create optimizer and scheduler
Epoch: 0


/root/Semi-supervised-learning/semilearn/nets/vit/vit.py:36: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(H == self.img_size[0], f"Input image height ({H}) doesn't match model ({self.img_size[0]}).")
/root/Semi-supervised-learning/semilearn/nets/vit/vit.py:37: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(W == self.img_size[1], f"Input image width ({W}) doesn't match model ({self.img_size[1]}).")
/root/Semi-supervised-learning/semilearn/nets/vit/vit.py:36: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(H == self.img_size[0], f"Input image height ({H}) doesn't match model ({self.img_size[0]}).")
/root/Semi-supervised-learning/semilearn/nets/vit/vit.py:37: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(W == self.img_size[1], f"Input image width ({W}) doesn't match model ({self.img_size[1]}).")


RuntimeError: CUDA out of memory. Tried to allocate 182.00 MiB (GPU 0; 10.75 GiB total capacity; 9.19 GiB already allocated; 168.12 MiB free; 9.33 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

### 중간 경과
Remixmatch를 진행하며 runtime error가 발생하여 remixmatch, fixmatch, flexmatch에 대한 코드를 gpu 메모리를 비우고 다시 진행하였습니다.

In [10]:
# 10 epcoh으로 실험 진행

epochs = 10
algorithm_list = ['remixmatch','fixmatch','flexmatch']
num_labels_list = [40]
for algorithm in algorithm_list:
    for num_labels in num_labels_list:
        train_lb_loader, train_ulb_loader, eval_loader = dataloader(algorithm, num_labels)
        trainer = train(algorithm, num_labels, train_lb_loader, train_ulb_loader, eval_loader, epochs)
        globals()["eval_dict_{}_{}".format(algorithm, num_labels)] = trainer.evaluate(eval_loader)
        globals()["y_pred_{}_{}".format(algorithm, num_labels)], globals()["y_logit_{}_{}".format(algorithm, num_labels)]= trainer.predict(eval_loader)

/bin/sh: 1: netstat: not found


Files already downloaded and verified
lb count: [4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
ulb count: [5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000]
Files already downloaded and verified


/bin/sh: 1: netstat: not found


Files already downloaded and verified
lb count: [4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
ulb count: [5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000]
Files already downloaded and verified
unlabeled data number: 50000, labeled data number 40
Create train and test data loaders
[!] data loader keys: dict_keys(['train_lb', 'train_ulb', 'eval'])
_IncompatibleKeys(missing_keys=['head.weight', 'head.bias'], unexpected_keys=[])
Create optimizer and scheduler
distribution alignment p_target: tensor([0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
        0.1000], dtype=torch.float64)
Epoch: 0


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
[2022-12-26 09:38:39,320 INFO] confusion matrix
[2022-12-26 09:38:39,321 INFO] [[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]
[2022-12-26 09:38:39,322 INFO] evaluation metric
[2022-12-26 09:38:39,322 INFO] acc: 0.1000
[2022-12-26 09:38:39,323 INFO] precision: 0.0100
[2022-12-26 09:38:39,323 INFO] recall: 0.1000
[2022-12-26 09:38:39,323 INFO] f1: 0.0182


...

Files already downloaded and verified
lb count: [4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
ulb count: [5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000]
Files already downloaded and verified


/bin/sh: 1: netstat: not found


Files already downloaded and verified
lb count: [4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
ulb count: [5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000]
Files already downloaded and verified
unlabeled data number: 50000, labeled data number 40
Create train and test data loaders
[!] data loader keys: dict_keys(['train_lb', 'train_ulb', 'eval'])
_IncompatibleKeys(missing_keys=['head.weight', 'head.bias'], unexpected_keys=[])
Create optimizer and scheduler
Epoch: 0


/root/Semi-supervised-learning/semilearn/nets/vit/vit.py:36: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(H == self.img_size[0], f"Input image height ({H}) doesn't match model ({self.img_size[0]}).")
/root/Semi-supervised-learning/semilearn/nets/vit/vit.py:37: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(W == self.img_size[1], f"Input image width ({W}) doesn't match model ({self.img_size[1]}).")
/root/Semi-supervised-learning/semilearn/nets/vit/vit.py:36: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(H == self.img_size[0], f"Input image height ({H}) doesn't match model ({self.img_size[0]}).")
/root/Semi-supervised-learning/semilearn/nets/vit/vit.py:37: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(W == self.img_size[1], f"Input image width ({W}) doesn't match model ({self.img_size[1]}).")
/root/Semi-supervised-learning/semilearn/nets/vit/vit.py:36: SyntaxWar

RuntimeError: CUDA out of memory. Tried to allocate 182.00 MiB (GPU 0; 10.75 GiB total capacity; 9.30 GiB already allocated; 139.75 MiB free; 9.35 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

### 실험 중간 경과
FixMatch와 FlexMatch의 경우 batch size를 16으로 진행하였을때 run out of memory error가 발생하여 batch size를 8로 수정하고 실험을 진행하겠습니다.

In [19]:
# 10 epcoh으로 실험 진행

epochs = 10
algorithm_list = ['fixmatch','flexmatch']
num_labels_list = [40]
for algorithm in algorithm_list:
    for num_labels in num_labels_list:
        train_lb_loader, train_ulb_loader, eval_loader = dataloader(algorithm, num_labels)
        trainer = train(algorithm, num_labels, train_lb_loader, train_ulb_loader, eval_loader, epochs)
        globals()["eval_dict_{}_{}".format(algorithm, num_labels)] = trainer.evaluate(eval_loader)
        globals()["y_pred_{}_{}".format(algorithm, num_labels)], globals()["y_logit_{}_{}".format(algorithm, num_labels)]= trainer.predict(eval_loader)

/bin/sh: 1: netstat: not found


Files already downloaded and verified
lb count: [4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
ulb count: [5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000]
Files already downloaded and verified


/bin/sh: 1: netstat: not found


Files already downloaded and verified
lb count: [4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
ulb count: [5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000]
Files already downloaded and verified
unlabeled data number: 50000, labeled data number 40
Create train and test data loaders
[!] data loader keys: dict_keys(['train_lb', 'train_ulb', 'eval'])
_IncompatibleKeys(missing_keys=['head.weight', 'head.bias'], unexpected_keys=[])
Create optimizer and scheduler
Epoch: 0


KeyboardInterrupt: 

### 실험 결과 분석

각 알고리즘의 labeled data를 class당 4개씩 사용하였을때의 semi-supervised learning의 결과는 다음과 같습니다.  

<img src="ssl_pictures/results_all.png" width="708px" height="257px" title="RESULT">  

위의 표에서 알 수 있듯이, MixMatch가 가장 높은 성능을 기록하였습니다. 다만 모든 알고리즘에서 성능이 상대적으로 낮은 것을 확인 할 수 있었습니다. 높은 확률로 학습 epoch이 작기 때문에 발생한 문제로 예상됩니다. 다만 시간적, 장치적 한계로 인해 Microsoft 코드 상에서 제안하는 학습 epoch을 진행하기에는 무리가 있었습니다. 조금 더 시간적, 장치적 여유를 갖고 실험을 진행하였다면 더 높은 성능을 얻을 수 있을 것 이라 판단됩니다.

우선은 실험에서 가장 높은 성능을 보인 **MixMatch**에 대해 labeled data의 수를 바꿔가며 추가 실험을 진행하도록 하겠습니다.  





In [22]:
# num_labels = 250

epochs = 10
algorithm_list = ['mixmatch']
num_labels_list = [250]
for algorithm in algorithm_list:
    for num_labels in num_labels_list:
        train_lb_loader, train_ulb_loader, eval_loader = dataloader(algorithm, num_labels)
        trainer = train(algorithm, num_labels, train_lb_loader, train_ulb_loader, eval_loader, epochs)
        globals()["eval_dict_{}_{}".format(algorithm, num_labels)] = trainer.evaluate(eval_loader)
        globals()["y_pred_{}_{}".format(algorithm, num_labels)], globals()["y_logit_{}_{}".format(algorithm, num_labels)]= trainer.predict(eval_loader)

/bin/sh: 1: netstat: not found


Files already downloaded and verified
lb count: [25, 25, 25, 25, 25, 25, 25, 25, 25, 25]
ulb count: [5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000]
Files already downloaded and verified


/bin/sh: 1: netstat: not found


Files already downloaded and verified
lb count: [25, 25, 25, 25, 25, 25, 25, 25, 25, 25]
ulb count: [5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000]
Files already downloaded and verified
unlabeled data number: 50000, labeled data number 250
Create train and test data loaders
[!] data loader keys: dict_keys(['train_lb', 'train_ulb', 'eval'])
_IncompatibleKeys(missing_keys=['head.weight', 'head.bias'], unexpected_keys=[])
Create optimizer and scheduler
Epoch: 0


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
[2022-12-27 04:05:16,451 INFO] confusion matrix
[2022-12-27 04:05:16,451 INFO] [[0.474 0.095 0.    0.305 0.038 0.    0.01  0.    0.078 0.   ]
 [0.17  0.303 0.    0.5   0.006 0.    0.015 0.    0.006 0.   ]
 [0.083 0.049 0.    0.63  0.065 0.    0.165 0.    0.008 0.   ]
 [0.029 0.045 0.    0.807 0.032 0.    0.087 0.    0.    0.   ]
 [0.07  0.022 0.    0.504 0.088 0.    0.312 0.    0.004 0.   ]
 [0.032 0.043 0.    0.834 0.024 0.    0.067 0.    0.    0.   ]
 [0.022 0.015 0.    0.563 0.033 0.    0.367 0.    0.    0.   ]
 [0.033 0.07  0.    0.829 0.017 0.    0.05  0.    0.001 0.   ]
 [0.465 0.15  0.    0.291 0.017 0.    0.004 0.    0.073 0.   ]
 [0.183 0.232 0.    0.561 0.007 0.    0.01  0.    0.007 

...

model saved: ./saved_models/classic_cv/mixmatch_cifar10_250_0/latest_model.pth
model saved: ./saved_models/classic_cv/mixmatch_cifar10_250_0/model_best.pth
Epoch: 9


[2022-12-27 06:17:48,525 INFO] confusion matrix
[2022-12-27 06:17:48,525 INFO] [[0.35  0.081 0.02  0.131 0.032 0.024 0.009 0.028 0.165 0.16 ]
 [0.111 0.401 0.01  0.077 0.005 0.02  0.008 0.041 0.031 0.296]
 [0.129 0.081 0.091 0.208 0.106 0.144 0.06  0.073 0.035 0.073]
 [0.053 0.079 0.015 0.455 0.04  0.159 0.038 0.051 0.022 0.088]
 [0.083 0.032 0.027 0.184 0.316 0.085 0.085 0.116 0.034 0.038]
 [0.069 0.09  0.018 0.36  0.051 0.219 0.02  0.087 0.025 0.061]
 [0.053 0.033 0.023 0.309 0.11  0.072 0.29  0.041 0.013 0.056]
 [0.062 0.054 0.024 0.147 0.07  0.113 0.016 0.389 0.014 0.111]
 [0.184 0.097 0.004 0.078 0.021 0.027 0.008 0.017 0.376 0.188]
 [0.083 0.183 0.004 0.062 0.008 0.013 0.009 0.049 0.057 0.532]]
[2022-12-27 06:17:48,527 INFO] evaluation metric
[2022-12-27 06:17:48,527 INFO] acc: 0.3419
[2022-12-27 06:17:48,527 INFO] precision: 0.3719
[2022-12-27 06:17:48,528 INFO] recall: 0.3419
[2022-12-27 06:17:48,528 INFO] f1: 0.3360
[2022-12-27 06:17:48,716 INFO] Best acc 0.3437 at epoch 8
[20

model saved: ./saved_models/classic_cv/mixmatch_cifar10_250_0/latest_model.pth


[2022-12-27 06:25:57,486 INFO] confusion matrix
[2022-12-27 06:25:57,487 INFO] [[0.35  0.081 0.02  0.131 0.032 0.024 0.009 0.028 0.165 0.16 ]
 [0.111 0.401 0.01  0.077 0.005 0.02  0.008 0.041 0.031 0.296]
 [0.129 0.081 0.091 0.208 0.106 0.144 0.06  0.073 0.035 0.073]
 [0.053 0.079 0.015 0.455 0.04  0.159 0.038 0.051 0.022 0.088]
 [0.083 0.032 0.027 0.184 0.316 0.085 0.085 0.116 0.034 0.038]
 [0.069 0.09  0.018 0.36  0.051 0.219 0.02  0.087 0.025 0.061]
 [0.053 0.033 0.023 0.309 0.11  0.072 0.29  0.041 0.013 0.056]
 [0.062 0.054 0.024 0.147 0.07  0.113 0.016 0.389 0.014 0.111]
 [0.184 0.097 0.004 0.078 0.021 0.027 0.008 0.017 0.376 0.188]
 [0.083 0.183 0.004 0.062 0.008 0.013 0.009 0.049 0.057 0.532]]
[2022-12-27 06:25:57,488 INFO] evaluation metric
[2022-12-27 06:25:57,489 INFO] acc: 0.3419
[2022-12-27 06:25:57,489 INFO] precision: 0.3719
[2022-12-27 06:25:57,490 INFO] recall: 0.3419
[2022-12-27 06:25:57,490 INFO] f1: 0.3360


KeyboardInterrupt: 

In [8]:
# num_labels = 4,000

epochs = 10
algorithm_list = ['mixmatch']
num_labels_list = [4000]
for algorithm in algorithm_list:
    for num_labels in num_labels_list:
        train_lb_loader, train_ulb_loader, eval_loader = dataloader(algorithm, num_labels)
        trainer = train(algorithm, num_labels, train_lb_loader, train_ulb_loader, eval_loader, epochs)
        globals()["eval_dict_{}_{}".format(algorithm, num_labels)] = trainer.evaluate(eval_loader)
        globals()["y_pred_{}_{}".format(algorithm, num_labels)], globals()["y_logit_{}_{}".format(algorithm, num_labels)]= trainer.predict(eval_loader)

/bin/sh: 1: netstat: not found


Files already downloaded and verified
lb count: [400, 400, 400, 400, 400, 400, 400, 400, 400, 400]
ulb count: [5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000]
Files already downloaded and verified


/bin/sh: 1: netstat: not found


Files already downloaded and verified
lb count: [400, 400, 400, 400, 400, 400, 400, 400, 400, 400]
ulb count: [5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000]
Files already downloaded and verified
unlabeled data number: 50000, labeled data number 4000
Create train and test data loaders
[!] data loader keys: dict_keys(['train_lb', 'train_ulb', 'eval'])
_IncompatibleKeys(missing_keys=['head.weight', 'head.bias'], unexpected_keys=[])
Create optimizer and scheduler
Epoch: 0


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
[2022-12-27 10:43:18,936 INFO] confusion matrix
[2022-12-27 10:43:18,937 INFO] [[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
[2022-12-27 10:43:18,938 INFO] evaluation metric
[2022-12-27 10:43:18,939 INFO] acc: 0.1000
[2022-12-27 10:43:18,939 INFO] precision: 0.0100
[2022-12-27 10:43:18,940 INFO] recall: 0.1000
[2022-12-27 10:43:18,940 INFO] f1: 0.0182


...

[2022-12-27 12:06:14,996 INFO] confusion matrix
[2022-12-27 12:06:14,997 INFO] [[0.12103934 0.14102083 0.00300098 0.00500163 0.0040013  0.0280091
  0.01500488 0.01400455 0.44296896 0.22594843]
 [0.00600195 0.45202191 0.         0.00100033 0.         0.01700553
  0.00500163 0.00300098 0.31797834 0.19798935]
 [0.0260091  0.18596509 0.0180063  0.02700945 0.039989   0.23898364
  0.21002351 0.059996   0.03901365 0.15500425]
 [0.0050025  0.20195098 0.0030015  0.01698349 0.02301151 0.42896448
  0.07201101 0.07103552 0.0190095  0.15902951]
 [0.0080028  0.14300005 0.00897814 0.0280098  0.08295403 0.25096284
  0.31405992 0.04301506 0.0340119  0.08700545]
 [0.0030009  0.1150095  0.0030009  0.0160048  0.0250075  0.55706712
  0.0549915  0.06694508 0.0180054  0.14096729]
 [0.00300053 0.2119871  0.00900158 0.03795664 0.05798515 0.20201035
  0.38304203 0.02500438 0.0080014  0.06201085]
 [0.0140042  0.1189857  0.0010003  0.0190057  0.0220066  0.39501851
  0.0370111  0.10998299 0.0250075  0.25797739]
 [

model saved: ./saved_models/classic_cv/mixmatch_cifar10_4000_0/latest_model.pth
model saved: ./saved_models/classic_cv/mixmatch_cifar10_4000_0/model_best.pth
Epoch: 9


[2022-12-27 12:16:38,173 INFO] confusion matrix
[2022-12-27 12:16:38,174 INFO] [[0.36501863 0.12599095 0.0080026  0.         0.01500488 0.02300748
  0.02500813 0.02100683 0.25605822 0.16090229]
 [0.01000325 0.59711906 0.00100033 0.00097532 0.00100033 0.0200065
  0.01700553 0.00100033 0.11996399 0.23192538]
 [0.08402941 0.11994198 0.01100385 0.0060021  0.0760016  0.26894413
  0.25403891 0.05699495 0.02300805 0.10003501]
 [0.02701351 0.14394697 0.0050025  0.0070035  0.02101051 0.47396198
  0.11203102 0.0809905  0.008004   0.12103552]
 [0.03701295 0.0760016  0.01100385 0.00997849 0.10296104 0.28197369
  0.35604962 0.04601611 0.0140049  0.06499775]
 [0.0160048  0.0960038  0.0020006  0.0070021  0.0190057  0.6120086
  0.0769981  0.0589927  0.0060018  0.10598179]
 [0.01400245 0.1120196  0.01000175 0.00697622 0.05901033 0.18500738
  0.53396844 0.02200385 0.0040007  0.05300928]
 [0.0180054  0.0860008  0.0040012  0.0040012  0.0250075  0.48001901
  0.06196859 0.12396219 0.0050015  0.19203261]
 [0

model saved: ./saved_models/classic_cv/mixmatch_cifar10_4000_0/latest_model.pth
model saved: ./saved_models/classic_cv/mixmatch_cifar10_4000_0/model_best.pth


[2022-12-27 12:20:42,870 INFO] confusion matrix
[2022-12-27 12:20:42,871 INFO] [[0.36501863 0.12599095 0.0080026  0.         0.01500488 0.02300748
  0.02500813 0.02100683 0.25605822 0.16090229]
 [0.01000325 0.59711906 0.00100033 0.00097532 0.00100033 0.0200065
  0.01700553 0.00100033 0.11996399 0.23192538]
 [0.08402941 0.11994198 0.01100385 0.0060021  0.0760016  0.26894413
  0.25403891 0.05699495 0.02300805 0.10003501]
 [0.02701351 0.14394697 0.0050025  0.0070035  0.02101051 0.47396198
  0.11203102 0.0809905  0.008004   0.12103552]
 [0.03701295 0.0760016  0.01100385 0.00997849 0.10296104 0.28197369
  0.35604962 0.04601611 0.0140049  0.06499775]
 [0.0160048  0.0960038  0.0020006  0.0070021  0.0190057  0.6120086
  0.0769981  0.0589927  0.0060018  0.10598179]
 [0.01400245 0.1120196  0.01000175 0.00697622 0.05901033 0.18500738
  0.53396844 0.02200385 0.0040007  0.05300928]
 [0.0180054  0.0860008  0.0040012  0.0040012  0.0250075  0.48001901
  0.06196859 0.12396219 0.0050015  0.19203261]
 [0

### Labeled data 수에 따른 MixMatch 성능 비교

Epoch = 10 을 기준으로 num_labels = 40, 250, 4000 일때의 성능에 대한 비교를 진행하였고 결과는 다음과 같습니다.  

<img src="ssl_pictures/num_labels_results.png" width="708px" height="153px" title="NUM_Labels_RESULT">  

일반적으로 labeled data가 많을수록 Semi-supervised learning의 성능이 올라가지만, 본 실험의 경우 num_labels = 250인 경우가 4,000 인 경우보다 높게 성능이 측정되었습니다. 첫번째 실험 결과 분석에서도 언급했듯이, 학습 epoch이 충분하지 않기 때문에 발생한 현상이라고 판단됩니다. 실제로 num_labels = 4000 의 학습 log를 살펴보면 4 epoch까지는 모든 data를 1번 class로 예측하는 것을 확인 할 수 있습니다. 만약 학습 epoch을 상승 시켜 다시 실험을 진행한다면, 조금 더 높은 성능을 기록할 것이라고 예상됩니다.  

